In [53]:
import pandas as pd

In [54]:
import duckdb

query = """SELECT year, county,state_name, county_fips,
SUM("502_investment_dollars") as total_502_investment_dollars,
SUM("number_of_502_investment") as total_number_of_502_investment
FROM './final_data/*.csv'
GROUP BY year, county, state_name, county_fips
ORDER BY year, state_name, county
;
"""

with duckdb.connect() as conn:
    result = conn.sql(query)

    df = result.fetchdf()

    
    # Join the county geomoetries with the aggregated data
    # Create a geojson with the geometries and aggregated data
    # Save the geojson to a file
    # Display the map with folium

print(df.head())



InvalidInputException: Invalid Input Error: Schema mismatch between globbed files.
Main file schema: .\final_data\2015.csv
Current file: .\final_data\alabama2015metadata.csv
Column with name: "year" is missing
Column with name: "county" is missing
Column with name: "state_name" is missing
Column with name: "county_fips" is missing
Column with name: "502_investment_dollars" is missing
Column with name: "number_of_502_investment" is missing
Potential Fixes 
* Consider setting union_by_name=true.
* Consider setting files_to_sniff to a higher value (e.g., files_to_sniff = -1)


In [55]:
df.to_csv("aggregator.csv", index=False)
print("Saved aggregator.csv")

Saved aggregator.csv


In [56]:
df.head()

,year,county,state_name,county_fips,total_502_investment_dollars,total_number_of_502_investment
0,2015,Bibb,Alabama,01007,1273871.0,10.0
1,2015,Blount,Alabama,01009,8956456.0,75.0
2,2015,Calhoun,Alabama,01015,11578406.0,105.0
3,2015,Chambers,Alabama,01017,3911636.0,44.0
4,2015,Cherokee,Alabama,01019,2781056.0,27.0


In [57]:
import geopandas as gpd

# Read the shapefile
gdf = gpd.read_file('C:/Users/athar/OneDrive/Desktop/fulton_ring/cb_2018_us_county_5m/cb_2018_us_county_5m.shp')




In [58]:
print(gdf.head())


  STATEFP COUNTYFP  COUNTYNS        AFFGEOID  GEOID      NAME LSAD  \
0      39      071  01074048  0500000US39071  39071  Highland   06   
1      06      003  01675840  0500000US06003  06003    Alpine   06   
2      12      033  00295737  0500000US12033  12033  Escambia   06   
3      17      101  00424252  0500000US17101  17101  Lawrence   06   
4      28      153  00695797  0500000US28153  28153     Wayne   06   

        ALAND     AWATER                                           geometry  
0  1432479992   12194983  POLYGON ((-83.86976 39.05553, -83.86568 39.247...  
1  1912292630   12557304  POLYGON ((-120.07248 38.50987, -120.07239 38.7...  
2  1701544502  563927612  POLYGON ((-87.62999 30.87766, -87.62946 30.880...  
3   963936864    5077783  POLYGON ((-87.91028 38.57493, -87.90811 38.850...  
4  2099745573    7255476  POLYGON ((-88.94317 31.78421, -88.94336 31.824...  


In [59]:
df['county_fips'] = df['county_fips'].astype(str)

df['county_fips'] = df['county_fips'].str.zfill(5)



In [60]:
# Merge and sort so Alabama/Bibb is first (as in the CSV)
merged_gdf = gdf.merge(df, left_on='GEOID', right_on='county_fips')
cols_to_drop = ['STATEFP', 'COUNTYFP', 'COUNTYNS', 'AFFGEOID', 'NAME', 'LSAD', 'ALAND', 'AWATER']
merged_gdf = merged_gdf.drop(columns=cols_to_drop)
merged_gdf.columns = [col.lower() for col in merged_gdf.columns]
merged_gdf = merged_gdf.rename(columns={'geoid': 'geo_id'})

# Sort by state_name and county to match the original CSV order
merged_gdf = merged_gdf.sort_values(['state_name', 'county']).reset_index(drop=True)

merged_gdf.head()

,geo_id,geometry,year,county,state_name,county_fips,total_502_investment_dollars,total_number_of_502_investment
0,01007,"POLYGON ((-87.42194 33.00338, -87.31854 33.006...",2015,Bibb,Alabama,01007,1273871.0,10.0
1,01007,"POLYGON ((-87.42194 33.00338, -87.31854 33.006...",2016,Bibb,Alabama,01007,658855.0,7.0
2,01007,"POLYGON ((-87.42194 33.00338, -87.31854 33.006...",2017,Bibb,Alabama,01007,1710111.0,15.0
3,01007,"POLYGON ((-87.42194 33.00338, -87.31854 33.006...",2018,Bibb,Alabama,01007,2219547.0,16.0
4,01007,"POLYGON ((-87.42194 33.00338, -87.31854 33.006...",2019,Bibb,Alabama,01007,1596396.0,13.0


In [62]:
merged_gdf.to_csv('./final_data/final_aggregator.csv', index=False)


In [61]:
merged_gdf.to_file('./final_data/aggregator.geojson', driver='GeoJSON')